**id**: unique id for a news article

**title**: the title of a news article

**author**: author of the news article

**text**: the text of the article; could be incomplete

**label**: a label that marks the article as potentially unreliable

1: unreliable     0: reliable

In [119]:
import numpy as np
import pandas as pd
import re  #searching the text in a document
from nltk.corpus import stopwords   #nltk: natural language toolkit, corpus: the body of that particular text, the important content of the text
# stopwords: those words that doesn't add much value to the contect of the text like articales or (what, where...)
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer  #to convert the text into feature vectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

**PorterStemmer** stripping suffixes or prefixes to derive the root word. This can help in normalizing text data and reducing the dimensionality of features in machine learning tasks involving text analysis

First let's download the stopwords 

In [120]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\legion\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Those stopwords are unnecessary in our text; therefore, we will eliminate them.

### Data Pre-processing

In [121]:
# Loading the news dataset
df = pd.read_csv('train.csv')

In [122]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [123]:
df.shape

(20800, 5)

In [124]:
df["label"].value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [125]:
# Counting the number of missing values
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [126]:
# replacing the null values to empty string
df = df.fillna("")

In [127]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

#### Let's attempt prediction using only the 'title' and 'author' columns to check if they yield satisfactory results, considering that the 'text' column is quite extensive

In [128]:
# Mergin the author name and news title
df['content'] = df['author']+' '+df['title']
print(df['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [129]:
X = df.drop(columns='label', axis=1)
y = df['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

In [130]:
print(X)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Reminder=> **Stemming**: stripping suffixes or prefixes to derive the root word. This can help in normalizing text data and reducing the dimensionality of features in machine learning tasks involving text analysis

In [131]:
port_stem = PorterStemmer()

In [132]:
# Create a stemming function
def stemming(content):
    # Replace any character that is not within the range of 'a' to 'z' or 'A' to 'Z' with a space (' ').
    stemmed_content = re.sub(r'[^a-zA-Z]', ' ', content)
    # Convert the text to lowercase.
    stemmed_content = stemmed_content.lower()
    # Devide the text to words in a list
    stemmed_content = stemmed_content.split()
    # Stem the words by transforming prefixes and suffixes to their base forms and then remove stopwords, which do not add significant value to the text.
    stemmed_content = [ port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english') ]
    # Turn the list to normal text
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [133]:
# apply() is used to apply a function to each column or row of a DataFrame automaticaly or with using axis=0 or 1.
df['content'] = df['content'].apply(stemming)

In [134]:
print(df['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


#### Seperating Data and label

In [135]:
# .values" returns a two-dimensional NumPy array containing the DataFrame's values
X = df['content'].values
y = df['label'].values

In [136]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [137]:
print(y)

[1 0 1 ... 0 1 1]


### Converting the textual data to numerical data

**Tokenization** and **Vectorization**:

**Tokenization**: It tokenizes text data, splitting it into individual words or terms (tokens).
**Vectorization**: It converts the tokenized text into numerical representations suitable for machine learning algorithms.
Calculates TF-IDF Values:

**Term Frequency (TF)**: It measures how frequently a term occurs in a document. It's calculated as the number of times a term appears in a document divided by the total number of terms in the document.

**Inverse Document Frequency (IDF)**: It measures the importance of a term across a collection of documents. It's calculated as the logarithm of the total number of documents divided by the number of documents containing the term.

**TF-IDF**: It combines TF and IDF, giving higher weight to terms that are frequent within a document but rare across the entire document collection.

In [138]:
vectorizer = TfidfVectorizer()

In [139]:
vectorizer.fit(X)

TfidfVectorizer()

In [140]:
X = vectorizer.transform(X)

In [141]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

### Splitting the dataset to Training and Test

In [142]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=10)
print(X.shape, X_train.shape, X_test.shape)

(20800, 17128) (16640, 17128) (4160, 17128)


### Training the model: Logistic Regression

**Logistic Regression** is well-suited for problems with binary outcomes, such as spam detection (**Fake** or **Not fake**).

In [143]:
model = LogisticRegression()

In [144]:
model.fit(X_train, y_train)

LogisticRegression()

### Evaluation

#### acurracy score

In [145]:
# accuracy score on Training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)
print("Accuracy score of the training data : ", training_data_accuracy)

Accuracy score of the training data :  0.9861778846153846


In [146]:
# accuracy score on Test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)
print("Accuracy score of the test data : ", test_data_accuracy)

Accuracy score of the test data :  0.9798076923076923


### Building a Predictive System

In [150]:
print(X_test[0])

  (0, 11478)	0.5816613804304052
  (0, 7424)	0.4349005104175782
  (0, 5233)	0.34308865677845996
  (0, 1274)	0.45190557045723173
  (0, 961)	0.38807616975795045


In [167]:
X_new = X_test[77]
result = model.predict(X_new)
if result == 1:
    print("The news is fake")
else:
    print("The news is not fake")

The news is fake


In [168]:
y_test[77]

1